<a href="https://colab.research.google.com/github/lesleycolgan/spinecounting/blob/main/Spine_Density_Network_Final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Imports

In [ ]:
%%shell

pip install cython
# Install pycocotools, the version by default in Colab
# has a bug fixed in https://github.com/cocodataset/cocoapi/pull/354
pip install -U 'git+https://github.com/cocodataset/cocoapi.git#subdirectory=PythonAPI'
pip install torchmetrics
pip install gdown
pip install -r https://raw.githubusercontent.com/ultralytics/yolov5/master/requirements.txt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/cocodataset/cocoapi.git to /tmp/pip-req-build-nz9mt1ki
  Running command git clone --filter=blob:none --quiet https://github.com/cocodataset/cocoapi.git /tmp/pip-req-build-nz9mt1ki
  Resolved https://github.com/cocodataset/cocoapi.git to commit 8c9bcc3cf640524c4c20a9c40e89cb6a2f2fa0e9
  Preparing metadata (setup.py) ... done
  Created wheel for pycocotools: filename=pycocotools-2.0-cp38-cp38-linux_x86_64.whl size=421019 sha256=25c2e4609057ade7c9fb8f1da1ecb66bee0e498353f1bf838604a7b465d0d7d3
  Stored in directory: /tmp/pip-ephem-wheel-cache-kthg_g5b/wheels/56/da/49/cb71a7c450b59588934077f431100c05fbde50646ee84a8d40
Successfully built pycocotools
  Attempting uninstall: pycocotools
    Found existing installation: pycocotools 2.0.6
    Uninstalling pycocotools-2.0.

In [ ]:
!git clone https://github.com/Chancellor376/yolov5

fatal: destination path 'yolov5' already exists and is not an empty directory.


In [ ]:
!wget -O best.pt 'https://docs.google.com/uc?export=download&id=1-aJluFlqPJb3DrNboxlRICPuwUu_kTrD&confirm=t'

--2023-01-25 15:21:21--  https://docs.google.com/uc?export=download&id=1-aJluFlqPJb3DrNboxlRICPuwUu_kTrD&confirm=t
Resolving docs.google.com (docs.google.com)... 142.251.111.101, 142.251.111.138, 142.251.111.139, ...
Connecting to docs.google.com (docs.google.com)|142.251.111.101|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://doc-08-50-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/l04sno4idm9qiid18f94voavcvj4c9f9/1674660075000/16860881752403449149/*/1-aJluFlqPJb3DrNboxlRICPuwUu_kTrD?e=download&uuid=75d69e82-88c8-428c-972e-3a0437d6c713 [following]
--2023-01-25 15:21:21--  https://doc-08-50-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/l04sno4idm9qiid18f94voavcvj4c9f9/1674660075000/16860881752403449149/*/1-aJluFlqPJb3DrNboxlRICPuwUu_kTrD?e=download&uuid=75d69e82-88c8-428c-972e-3a0437d6c713
Resolving doc-08-50-docs.googleusercontent.com (doc-08-50-docs.googleusercontent.com)... 142.251.111.

In [ ]:
import os
import numpy as np
import torch
import torch.nn as nn
import torchvision
import torch.utils.data
import torchvision.datasets as dset
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection.mask_rcnn import MaskRCNNPredictor
from PIL import Image
import json
from google.colab import drive
import cv2
from google.colab.patches import cv2_imshow
from pycocotools.coco import COCO
from torchmetrics.detection.mean_ap import MeanAveragePrecision
from pprint import pprint
import requests
from pathlib import Path, PurePath
from glob import glob
import re
import matplotlib.pyplot as plt
import seaborn as sns
import shutil
Path.cwd()

PosixPath('/content')

In [ ]:
%matplotlib inline

In [ ]:
drive.mount('/content/drive')

Mounted at /content/drive


#Folder Path in Google Drive

In [ ]:
folder = input('Enter the folder path: ')
print(f'You entered {folder}')
folder = '/content/drive/MyDrive/' + folder
print(folder)
unit = input('Enter the conversion unit: ')
print(f'You entered {unit}')
conv = input('Enter the conversion factor (1 pixel = __ conversion units): ')
print(f'You entered {conv}')

#Model Predicitons

In [ ]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model = torch.hub.load('ultralytics/yolov5', 'custom', path='best.pt', force_reload=True)
model.to(device)

Downloading: "https://github.com/ultralytics/yolov5/zipball/master" to /root/.cache/torch/hub/master.zip
YOLOv5 🚀 2023-1-25 Python-3.8.10 torch-1.13.1+cu116 CUDA:0 (Tesla T4, 15110MiB)

Fusing layers... 
Model summary: 267 layers, 46108278 parameters, 0 gradients, 107.6 GFLOPs
Adding AutoShape... 


AutoShape(
  (model): DetectMultiBackend(
    (model): DetectionModel(
      (model): Sequential(
        (0): Conv(
          (conv): Conv2d(3, 64, kernel_size=(6, 6), stride=(2, 2), padding=(2, 2))
          (act): SiLU(inplace=True)
        )
        (1): Conv(
          (conv): Conv2d(64, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
          (act): SiLU(inplace=True)
        )
        (2): C3(
          (cv1): Conv(
            (conv): Conv2d(128, 64, kernel_size=(1, 1), stride=(1, 1))
            (act): SiLU(inplace=True)
          )
          (cv2): Conv(
            (conv): Conv2d(128, 64, kernel_size=(1, 1), stride=(1, 1))
            (act): SiLU(inplace=True)
          )
          (cv3): Conv(
            (conv): Conv2d(128, 128, kernel_size=(1, 1), stride=(1, 1))
            (act): SiLU(inplace=True)
          )
          (m): Sequential(
            (0): Bottleneck(
              (cv1): Conv(
                (conv): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1

In [ ]:
model.conf = 0.594
trans = torchvision.transforms.ToTensor()
trans2 = torchvision.transforms.ToPILImage()
p = Path(folder)
subfolders = [x for x in p.iterdir() if x.is_dir()]
subfolders = sorted(subfolders, key = lambda x: [int(k) if k.isdigit() else k for k in re.split('([0-9]+)', x.stem)])
total = []
for subdir in subfolders:
  imgs = []
  imgs.extend(Path(subdir).glob('*.tif'))
  images = []
  for file in imgs:
    i = Image.open(file)
    i = i.convert('RGB')
    images.append(i)
  sppixL = []
  length = []
  for im in images:
    spines = 0
    model.eval()
    #im = trans(im)
    CV1 = np.array(trans2(trans(im)), dtype=np.uint8)
    #imtest = trans(CV1)
    with torch.no_grad():
      prediction = model([im])
    spines += len(prediction.xyxy[0])
    if len(CV1.shape) == 3:
      CV1 = cv2.cvtColor(CV1, cv2.COLOR_BGR2GRAY)
    CV = cv2.blur(CV1, (10,10))
    ret, thresh = cv2.threshold(CV1, 1, 255, cv2.THRESH_OTSU)
    contours, heirarchy = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    contours_poly = [None]*len(contours)
    boundRect = [None]*len(contours)
    masks = []
    rect_areas = []
    for i, c in enumerate(contours):
      contours_poly[i] = cv2.approxPolyDP(c, 3, True)
      boundRect[i] = cv2.boundingRect(contours_poly[i])
      rect_areas.append(boundRect[i][2] * boundRect[i][3])
    avg_area = np.mean(rect_areas)
    lengths = []
    for i in boundRect:
      ind = boundRect.index(i)
      x1 = i[0]
      x2 = i[0] + i[2]
      y1 = i[1]
      y2 = i[1] + i[3]
      if rect_areas[ind] < 0.5 * avg_area:
        masks.append(contours_poly[ind])
        continue
      CVV = CV1.copy()
      listC = list(contours_poly)
      boxxx = cv2.boundingRect(contours_poly[ind])
      slength = cv2.arcLength(contours_poly[ind], True) / np.pi
      lengths.append(slength)
      listC.pop(ind)
    length.append(np.sum(lengths))
    sppix = spines/np.sum(lengths)
    sppixL.append(sppix)
    #print(spines)
  #print('Average Spine Density of Neuron ' + os.path.basename(os.path.normpath(subdir)) + ' (Spines over Dendrite Length in ' + str(unit) + '): ' + str(np.mean(sppixL) / float(conv)))
  total.append((os.path.basename(os.path.normpath(subdir)), (np.mean(sppixL) / float(conv))))

#Average Spine Density

In [ ]:
for i in total:
  print('Average Spine Density of Neuron ' + i[0] + ' (Spines over Dendrite Length in ' + str(unit) + '): ' + str(i[1]))